In [ ]:
#import statements
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import json
import math

import os
import codecs
from argparse import ArgumentParser
from urllib import request
from tqdm import trange
from joblib import Parallel, delayed

import requests, zipfile, io
import tarfile
import csv
from tqdm import tqdm

In [ ]:
download_url = "http://www.cs.cmu.edu/~dbamman/data/booksummaries.tar.gz"

In [ ]:
r = requests.get(download_url)

In [ ]:
tar_bytes = io.BytesIO(r.content)
tf = tarfile.open(mode = 'r', fileobj=tar_bytes)
tf.extractall()

In [ ]:
#code adapted from: https://www.kaggle.com/iamhungundji/book-summary-genre-prediction/notebook

data = []

with open("booksummaries/booksummaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in tqdm(reader):
        data.append(row)

16559it [00:00, 28966.15it/s]


In [ ]:
#code adapted from: https://www.kaggle.com/iamhungundji/book-summary-genre-prediction/notebook
wiki_id = []
freebase_id = []
book_title = []
book_author = []
publication_date = []
genre = []
summary = []

for i in tqdm(data):
    wiki_id.append(i[0])
    freebase_id.append(i[1])
    book_title.append(i[2])
    book_author.append(i[3])
    publication_date.append(i[4])
    genre.append(i[5])
    summary.append(i[6])

books_summary = pd.DataFrame(data, columns = {'wiki_id': wiki_id, 'freebase_id': freebase_id, 
                      'book_title': book_title, 'book_author': book_author,
                      'publication_date': publication_date, 'genre': genre, 
                      'summary': summary})
books_summary.head()

100%|██████████| 16559/16559 [00:00<00:00, 564878.29it/s]


,wiki_id,freebase_id,book_title,book_author,publication_date,genre,summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,,,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


In [ ]:
#download_url = 'http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip'
#!wget $download_url
#!unzip BX-CSV-Dump.zip

In [ ]:
#cover_cols =["ISBN","Book-Title","Book-Author","Year-Of-Publication","Publisher","Image-URL-S","Image-URL-M","Image-URL-L"]
#books_cover = pd.read_csv('BX-Books.csv', header=0, names=cover_cols, encoding='latin1', delimiter=';', error_bad_lines=False)
#books_cover.head()

In [ ]:
download_url = 'https://github.com/uchidalab/book-dataset/raw/master/Task2/book32-listing.csv'
!wget $download_url

--2021-02-15 17:41:08--  https://github.com/uchidalab/book-dataset/raw/master/Task2/book32-listing.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/uchidalab/book-dataset/master/Task2/book32-listing.csv [following]
--2021-02-15 17:41:08--  https://raw.githubusercontent.com/uchidalab/book-dataset/master/Task2/book32-listing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39483760 (38M) [text/plain]
Saving to: ‘book32-listing.csv’

book32-listing.csv  100%[===================>]  37.65M  65.7MB/s    in 0.6s    

2021-02-15 17:41:09 (65.7 MB/s) - ‘book32-listing.csv’ saved [39483760/39483760]

In [ ]:
#adapted from https://github.com/uchidalab/book-dataset/tree/master/scripts

csv_filepath = 'book32-listing.csv'
output_dirpath = 'covers'

header_names = ['Amazon ID (ASIN)', 'Filename', 'Image URL', 'Title', 'Author', 'Category ID',
                'Category']
with codecs.open(csv_filepath, mode='r', encoding='utf-8', errors='ignore') as f:
    csv = pd.read_csv(f, delimiter=",", header=None, names=header_names)

if not os.path.isdir(output_dirpath):
    os.makedirs(output_dirpath)

def download_image(i):
    filename = csv.iloc[i]['Filename']
    category = csv.iloc[i]['Category']
    inner_output_dirpath = os.path.join(output_dirpath, category)
    if not os.path.isdir(inner_output_dirpath):
        os.mkdir(inner_output_dirpath)
    output_filepath = os.path.join(inner_output_dirpath, filename)

    url = csv.iloc[i]['Image URL']
    if not os.path.isfile(output_filepath):
        downloaded_img = request.urlopen(url)
        f = open(output_filepath, mode='wb')
        f.write(downloaded_img.read())
        downloaded_img.close()
        f.close()

def download_all_images():
    print('[Download images into "{}"]'.format(args.output_dirpath))
    Parallel(n_jobs=-1)(delayed(download_image)(i) for i in trange(len(csv)))

In [ ]:
cover_cols = ['Amazon ID (ASIN)', 'Filename', 'Image URL', 'Title', 'Author', 'Category ID','Category']
books_cover = pd.read_csv(csv_filepath, header=None, names=cover_cols, encoding='latin1', error_bad_lines=False)
books_cover.head()

,Amazon ID (ASIN),Filename,Image URL,Title,Author,Category ID,Category
0,761183272,0761183272.jpg,http://ecx.images-amazon.com/images/I/61Y5cOdH...,Mom's Family Wall Calendar 2016,Sandra Boynton,3,Calendars
1,1623439671,1623439671.jpg,http://ecx.images-amazon.com/images/I/61t-hrSw...,Doug the Pug 2016 Wall Calendar,Doug the Pug,3,Calendars
2,B00O80WC6I,B00O80WC6I.jpg,http://ecx.images-amazon.com/images/I/41X-KQqs...,"Moleskine 2016 Weekly Notebook, 12M, Large, Bl...",Moleskine,3,Calendars
3,761182187,0761182187.jpg,http://ecx.images-amazon.com/images/I/61j-4gxJ...,365 Cats Color Page-A-Day Calendar 2016,Workman Publishing,3,Calendars
4,1578052084,1578052084.jpg,http://ecx.images-amazon.com/images/I/51Ry4Tsq...,Sierra Club Engagement Calendar 2016,Sierra Club,3,Calendars


In [ ]:
books_cover.shape

(271360, 8)

In [ ]:
books_cover.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [ ]:
combined_pd = books_summary
combined_pd = combined_pd.merge(books_cover, left_on=["book_title" ,"book_author"], right_on=["Title","Author"])

In [ ]:
combined_pd.shape
combined_pd.head()

,wiki_id,freebase_id,book_title,book_author,publication_date,genre,summary,Amazon ID (ASIN),Filename,Image URL,Title,Author,Category ID,Category
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca...",812911612X,812911612X.jpg,http://ecx.images-amazon.com/images/I/51rssg33...,Animal Farm,George Orwell,15,Literature & Fiction
1,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca...",151072558,0151072558.jpg,http://ecx.images-amazon.com/images/I/51wdIdct...,Animal Farm,George Orwell,27,Teen & Young Adult
2,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan...",393312836,0393312836.jpg,http://ecx.images-amazon.com/images/I/41Y5oe4r...,A Clockwork Orange,Anthony Burgess,24,Science Fiction & Fantasy
3,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan...",3453130790,3453130790.jpg,http://ecx.images-amazon.com/images/I/61BFz75O...,A Clockwork Orange,Anthony Burgess,15,Literature & Fiction
4,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...,679720219,0679720219.jpg,http://ecx.images-amazon.com/images/I/5189NK0O...,The Plague,Albert Camus,15,Literature & Fiction


In [ ]:
combined_pd_filtered = combined_pd.drop_duplicates(subset=['book_title', 'book_author'], keep='first')

In [ ]:
combined_pd_filtered.shape

(926, 14)

In [ ]:
combined_pd_filtered.head()

,wiki_id,freebase_id,book_title,book_author,publication_date,genre,summary,Amazon ID (ASIN),Filename,Image URL,Title,Author,Category ID,Category
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca...",812911612X,812911612X.jpg,http://ecx.images-amazon.com/images/I/51rssg33...,Animal Farm,George Orwell,15,Literature & Fiction
2,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan...",393312836,0393312836.jpg,http://ecx.images-amazon.com/images/I/41Y5oe4r...,A Clockwork Orange,Anthony Burgess,24,Science Fiction & Fantasy
4,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...,679720219,0679720219.jpg,http://ecx.images-amazon.com/images/I/5189NK0O...,The Plague,Albert Camus,15,Literature & Fiction
5,2152,/m/0x5g,All Quiet on the Western Front,Erich Maria Remarque,1929-01-29,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","The book tells the story of Paul Bäumer, a Ge...",449213943,0449213943.jpg,http://ecx.images-amazon.com/images/I/516yP3uD...,All Quiet on the Western Front,Erich Maria Remarque,15,Literature & Fiction
6,7923,/m/026l0,Dracula,Bram Stoker,1897,"{""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""...","The novel is told in epistolary format, as a ...",486411095,0486411095.jpg,http://ecx.images-amazon.com/images/I/51N33RX1...,Dracula,Bram Stoker,17,"Mystery, Thriller & Suspense"


In [ ]:
books_cover_filtered = books_cover.drop_duplicates(subset=['Title', 'Author'], keep='first')
books_cover_filtered.shape

(204906, 7)

In [ ]:
books_cover_filtered = books_cover_filtered.sort_values('Author')
books_cover_filtered.head()

,Amazon ID (ASIN),Filename,Image URL,Title,Author,Category ID,Category
73561,B00S1O283E,B00S1O283E.jpg,http://ecx.images-amazon.com/images/I/51ijae60...,Bitcoins for Beginners: Teach Me Everything I ...,30 Minute Reads,6,Computers & Technology
67847,B00TI458HE,B00TI458HE.jpg,http://ecx.images-amazon.com/images/I/41jQB7FK...,Adult Children of Alcoholics/Dysfunctional Fam...,Adult Children of Alcoholics/Dysfunctional Fa...,25,Self-Help
155649,B00XM7LYL2,B00XM7LYL2.jpg,http://ecx.images-amazon.com/images/I/61o%2BV3...,Religions of the World: The History and Belief...,Charles River Editors,21,Religion & Spirituality
113862,B010YVZ6IU,B010YVZ6IU.jpg,http://ecx.images-amazon.com/images/I/51N84SmB...,Financial Management Explained in 40 Minutes: ...,FastKnowledge,2,Business & Money
23575,B00OQP8XFO,B00OQP8XFO.jpg,http://ecx.images-amazon.com/images/I/61s2t4jK...,Banana Kong Game Guide,HiddenStuff Entertainment,13,Humor & Entertainment
